In [1]:
import gmaps
import json
import requests
import pandas as pd
import numpy as np
from citipy import citipy
import matplotlib.pyplot as plt
from scipy import stats
from config import api_key
from citipy import citipy

In [142]:
abnb_data = pd.read_csv("Resources/airbnb_data.csv")

/Users/alfonsocamero/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [141]:
abnb2 = abnb_data.drop('neighbourhood_group', axis=1)
abnb2_df = abnb2.drop(abnb2[abnb2['room_type'] == 'Private room'].index)
abnb2_df['city'] = abnb2_df['city'].replace({'Broward County':'Fort Lauderdale','Clark County':'Las Vegas','New York City':'New York','San Clara Country':'San Jose','San Mateo County':'San Mateo','Santa Cruz County':'Santa Cruz','Twin Cities MSA':'Twin Cities','Washington D.C.':'Washington'})
abnb_df = abnb2_df.drop(abnb2_df[(abnb2_df['city'] == 'Hawaii') | (abnb2_df['city'] == 'Rhode Island') | (abnb2_df['city'] == 'Twin Cities')].index)


In [139]:
Cities = []
for city in abnb_df['city']:
    if city not in Cities:
        Cities.append(city)

In [140]:
cities = [city.lower() for city in Cities]

In [134]:
states = ['NC', 'TX', 'MA', 'FL', 'MA', 'IL', 'NV', 'OH', 'CO', 'NJ', 'CA', 'TN', 'LA', 'NY', 'CA', 'CA', 'OR', 'OR', 'CA', 'CA', 'CA', 'CA', 'CA', 'WA', 'DC']

In [135]:
by_cities = abnb_df.groupby('city').mean().reset_index()
by_cities['city'] = cities
by_cities['City'] = Cities
by_cities['State'] = states

In [136]:
by_cities

,city,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,City,State
0,asheville,2.729089e+07,9.641011e+07,35.578463,-82.557080,217.851462,5.445029,76.460819,2.915155,6.272515,169.638596,Asheville,NC
1,austin,2.531386e+07,9.529217e+07,30.282802,-97.755954,311.782487,9.479961,33.144644,1.439457,49.934661,142.824630,Austin,TX
2,boston,2.787548e+07,1.070867e+08,42.343169,-71.080080,209.841602,34.919891,37.570778,1.625987,38.671825,184.854802,Boston,MA
3,fort lauderdale,2.040532e+07,5.844660e+07,42.371986,-71.107202,222.803670,39.790826,49.317431,1.513995,6.818349,136.440367,Fort Lauderdale,FL
4,cambridge,2.892406e+07,9.886834e+07,41.902314,-87.662216,179.361744,9.127082,42.281770,1.843966,17.318799,177.314855,Cambridge,MA
5,chicago,3.230840e+07,1.281696e+08,39.984240,-82.996481,395.042333,8.599247,47.689558,2.868683,11.855127,160.244591,Chicago,IL
6,las vegas,2.684450e+07,1.017304e+08,39.741899,-104.981299,174.750077,12.829403,48.564165,2.132750,9.085758,129.939663,Las Vegas,NV
7,columbus,2.957025e+07,1.145052e+08,26.098144,-80.138814,264.226556,7.100666,21.238818,1.339888,14.813383,202.122153,Columbus,OH
8,denver,2.791440e+07,9.991590e+07,40.726335,-74.052016,167.781026,10.259547,29.986277,1.543465,15.739857,186.286993,Denver,CO
9,jersey city,3.030031e+07,1.212235e+08,36.116909,-115.145809,243.013242,7.317966,32.129312,1.863386,27.664868,184.842406,Jersey City,NJ


In [145]:
for index, row in by_cities.iterrows():
    city = row["city"]
    City = row["City"]
    state = row["State"]
    url = f'http://public.opendatasoft.com/api/records/1.0/search/?dataset=cities-and-towns-of-the-united-states%40public&q={city}&facet=name&facet=state&refine.name={City}&refine.state={state}'
    response = requests.get(url).json()
    print(f"Searching population record for {city}")
    
    try:
        by_cities.loc[index, "Population"] = response['records'][0]['fields']['pop_2010']
    
        print(f"{city}'s population: {response['records'][0]['fields']['pop_2010']}")
    
        print('________________________________________________')
    
    except (KeyError, IndexError):
        print(f"Population for {city} not found... Continue scanning.")
        
        print('________________________________________________')
    
    #print(json.dumps(response, indent=4, sort_keys=True))
    #print(response['records'][0]['fields']['pop_2010'])

Searching population record for asheville
asheville's population: 83393
________________________________________________
Searching population record for austin
austin's population: 790390
________________________________________________
Searching population record for boston
boston's population: 617594
________________________________________________
Searching population record for fort lauderdale
fort lauderdale's population: 165521
________________________________________________
Searching population record for cambridge
cambridge's population: 105162
________________________________________________
Searching population record for chicago
chicago's population: 2695598
________________________________________________
Searching population record for las vegas
las vegas's population: 583756
________________________________________________
Searching population record for columbus
columbus's population: 787033
________________________________________________
Searching population record fo

In [138]:
by_cities

,city,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,City,State,Population
0,asheville,2.729089e+07,9.641011e+07,35.578463,-82.557080,217.851462,5.445029,76.460819,2.915155,6.272515,169.638596,Asheville,NC,83393
1,austin,2.531386e+07,9.529217e+07,30.282802,-97.755954,311.782487,9.479961,33.144644,1.439457,49.934661,142.824630,Austin,TX,790390
2,boston,2.787548e+07,1.070867e+08,42.343169,-71.080080,209.841602,34.919891,37.570778,1.625987,38.671825,184.854802,Boston,MA,617594
3,fort lauderdale,2.040532e+07,5.844660e+07,42.371986,-71.107202,222.803670,39.790826,49.317431,1.513995,6.818349,136.440367,Fort Lauderdale,FL,165521
4,cambridge,2.892406e+07,9.886834e+07,41.902314,-87.662216,179.361744,9.127082,42.281770,1.843966,17.318799,177.314855,Cambridge,MA,105162
5,chicago,3.230840e+07,1.281696e+08,39.984240,-82.996481,395.042333,8.599247,47.689558,2.868683,11.855127,160.244591,Chicago,IL,2695598
6,las vegas,2.684450e+07,1.017304e+08,39.741899,-104.981299,174.750077,12.829403,48.564165,2.132750,9.085758,129.939663,Las Vegas,NV,583756
7,columbus,2.957025e+07,1.145052e+08,26.098144,-80.138814,264.226556,7.100666,21.238818,1.339888,14.813383,202.122153,Columbus,OH,787033
8,denver,2.791440e+07,9.991590e+07,40.726335,-74.052016,167.781026,10.259547,29.986277,1.543465,15.739857,186.286993,Denver,CO,600158
9,jersey city,3.030031e+07,1.212235e+08,36.116909,-115.145809,243.013242,7.317966,32.129312,1.863386,27.664868,184.842406,Jersey City,NJ,247597
